In [1]:
from pydrake.all import FindResourceOrThrow, StartMeshcat, MeshcatVisualizer
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import time
from pydrake.math import RigidTransform
import pydot
from IPython.display import SVG, display


from pydrake.all import (
    DiagramBuilder, Simulator, FindResourceOrThrow, MultibodyPlant, PiecewisePolynomial, SceneGraph,
    Parser, MultibodyPositionToGeometryPose, TrajectorySource, Demultiplexer, ConstantVectorSource,
    AddMultibodyPlantSceneGraph, HalfSpace, CoulombFriction
)


def vis(x_traj, tf, time_step=0.1):
    # Create a MultibodyPlant for the arm
    file_name = "planar_walker.urdf"
    builder = DiagramBuilder()
    scene_graph = builder.AddSystem(SceneGraph())
    plant = MultibodyPlant(0.0)
    plant.RegisterAsSourceForSceneGraph(scene_graph)
    Parser(plant=plant).AddModels(file_name)

    plant.WeldFrames(
                plant.world_frame(),
                plant.GetBodyByName("base").body_frame(),
                RigidTransform.Identity()
            )

    plant.Finalize()

    n_q = plant.num_positions()
    n_v = plant.num_velocities()
    n_u = plant.num_actuators()

    x_traj_source = builder.AddSystem(TrajectorySource(x_traj))

    demux = builder.AddSystem(Demultiplexer(np.array([n_q, n_v])))
    to_pose = builder.AddSystem(MultibodyPositionToGeometryPose(plant))

    builder.Connect(x_traj_source.get_output_port(), demux.get_input_port())
    builder.Connect(demux.get_output_port(0), to_pose.get_input_port())
    builder.Connect(to_pose.get_output_port(), scene_graph.get_source_pose_port(plant.get_source_id()))


    MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

    diagram = builder.Build()
    diagram.set_name("diagram")

    simulator = Simulator(diagram)
    simulator.Initialize()
    simulator.set_target_realtime_rate(time_step)
    simulator.AdvanceTo(tf);

## Swing Leg

In [ ]:
import importlib
import traj_optim
import fsm_utils

importlib.reload(traj_optim)
importlib.reload(fsm_utils)
from traj_optim import TrajectoryOptimizationSolution, read_csv
from fsm_utils import (
    LEFT_STANCE, RIGHT_STANCE, SPACE_STANCE, DOUBLE_SUPPORT
)

# Walking

# Blue is Right feet, Red is left

solver = TrajectoryOptimizationSolution(n_foot=6)
N = 10
mode_seqs = [RIGHT_STANCE] #, LEFT_STANCE, RIGHT_STANCE]
repeat = 1
initial_states = read_csv("q&v.csv", N * repeat * len(mode_seqs))
tf = 0.5 # just now is tf = 2
destination = 0.3

x_traj_swing, u_traj_swing, prog = solver.solve(N, 
                                    mode_seqs, 
                                    repeat, 
                                    initial_states,
                                     mu=1, 
                                     tf=tf, 
                                     destination=destination, 
                                     iters=2e4,
                                     test=True)

In [ ]:
vis(x_traj_swing, tf, 0.1)

## Jumping

In [ ]:
#%tb

import importlib
import traj_optim
import fsm_utils

importlib.reload(traj_optim)
importlib.reload(fsm_utils)
from traj_optim import TrajectoryOptimizationSolution, read_csv
from fsm_utils import (
    LEFT_STANCE, RIGHT_STANCE, SPACE_STANCE, DOUBLE_SUPPORT
)


solver = TrajectoryOptimizationSolution(n_foot=6)
N = 6
mode_seqs = [DOUBLE_SUPPORT, SPACE_STANCE, DOUBLE_SUPPORT] #, LEFT_STANCE, RIGHT_STANCE]
repeat = 1
n_mode = len(mode_seqs)
#initial_states = read_csv("q&v.csv", N * repeat * len(mode_seqs))
initial_state = np.array(
    [0.00000000,0.80000000,0.00000000,
              -0.64350111,1.28700222,-0.64350111,1.28700222,
              0.20000000,0.00000000,0.00000000,0.00000000,
              0.00000000,0.00000000,0.00000000]
      )
initial_guesses = np.concatenate([initial_state for _ in range(n_mode * N * repeat)]).reshape(n_mode * N * repeat, 14)
initial_guesses[:, 0] = np.linspace(initial_state[0], destination, n_mode * N * repeat)

tf = 1.0 # just now is tf = 2
destination = 1.2

x_traj_jump, u_traj_jump, prog = solver.solve(N, 
                                    mode_seqs, 
                                    repeat, 
                                    initial_guesses,
                                     mu=5, 
                                     tf=tf, 
                                     destination=destination, 
                                     iters=5e4,
                                     test=False)

In [ ]:
vis(x_traj_jump, tf, time_step=0.1)

In [ ]:
x_traj_jump_repeat = np.array(
[[[ 6.12423115e-14,  8.00000000e-01 ,  1.12021434e-13 ,  -6.43501110e-01,
    1.28700222e+00,  -6.43501110e-01,   1.28700222e+00,   2.00000000e-01,
   -1.35175472e-15,  -3.74779571e-14,   2.30633957e-15,   4.41549641e-14,
   -6.49378178e-15,  -1.15769273e-19],
  [-5.78900843e-04,  7.90068205e-01, -6.16213829e-02 , -7.30569342e-01,
    1.31964482e+00, -7.30569342e-01,  1.31964482e+00 , -2.04770868e-01,
   -1.62443235e-01, -1.81419844e+00, -2.33718207e+00 ,  5.23895575e-01,
   -2.33718207e+00,  5.23895575e-01],
  [-2.79641615e-02,  7.80729923e-01, -2.45516203e-01 , -9.63516116e-01,
    1.34747475e+00, -9.63516116e-01,  1.34747475e+00 , -6.30936132e-01,
   -2.15786925e-01, -3.78899511e+00, -4.90854351e+00 ,  6.01553224e-01,
   -4.90854351e+00,  6.01553224e-01],
  [-7.91518655e-02,  7.54959910e-01, -5.48892589e-01 , -1.36977436e+00,
    1.41553364e+00, -1.36977436e+00,  1.41553364e+00 , -8.25564274e-01,
   -5.76937563e-01, -5.03918746e+00, -6.94288816e+00 ,  1.47681095e+00,
   -6.94288816e+00,  1.47681095e+00],
  [-1.30083976e-01,  7.16423735e-01, -8.89685645e-01 , -1.83170073e+00 ,
    1.50700049e+00, -1.83170073e+00,  1.50700049e+00 , -6.90293914e-01 ,
   -3.68829507e-01, -5.18654707e+00, -6.55151949e+00 ,  6.80898598e-01 ,
   -6.55151949e+00,  6.80898598e-01],
  [-1.75417339e-04,  8.58581808e-01, -1.04719755e+00 , -1.55094760e+00,
   -6.72052347e-02, -1.55094760e+00, -6.72052347e-02 , -4.13791405e-01,
   -5.39861350e-01, -5.42915665e+00,  1.43867670e+00 , -1.50000000e+01,
    1.43867670e+00, -1.50000000e+01]],

 [[-1.75417339e-04,  8.58581808e-01, -1.04719755e+00, -1.55094760e+00 ,
   -6.72052347e-02, -1.55094760e+00, -6.72052347e-02,  4.27994467e+00 ,
    2.74526193e+00,  4.84640929e+00,  1.50000000e+01,  1.50000000e+01 ,
    5.49780070e+00, -1.50000000e+01,],
  [ 2.68179583e-01,  1.01620703e+00, -8.22984081e-01, -2.00987331e+00,
    1.76628696e+00, -1.82748902e+00,  2.48577431e-02,  4.32237726e+00,
    2.03492743e+00,  4.03325113e+00,  1.50000000e+01,  1.50000000e+01,
   -8.50831658e+00,  7.59585501e+00,],
  [ 5.67586832e-01,  1.11010372e+00, -4.97689697e-01,  7.87653534e-01 ,
    3.10239538e+00, -2.28858721e+00,  1.63792341e-02,  4.10471050e+00 ,
    1.34614272e+00,  3.07509516e+00, -6.65237052e+00, -1.50000000e+01 ,
   -2.10356036e+00, -1.50000000e+01,],
  [ 7.77824547e-01,  1.15155886e+00, -4.66555706e-01, -9.59566874e-01 ,
   -1.64102144e+00, -2.27217676e+00, -8.20282561e-01,  2.34281563e+00 ,
   -5.44873906e-01, -4.26827115e-01,  2.51854819e+00, -8.13108868e+00 ,
    1.42806833e+00,  4.25793898e-02,],
  [ 8.70502697e-01,  1.03753455e+00, -5.64508137e-01,  4.94356404e-01 ,
    3.14159265e+00, -2.19173320e+00,  1.07817917e-01,  2.22806388e-01 ,
   -2.44668635e+00, -4.42495254e+00, -1.21631791e+01,  1.50000000e+01 ,
    2.10658640e+00,  1.50000000e+01,],
  [ 8.74649714e-01,  8.81176607e-01, -9.64851511e-01, -1.66747506e+00 ,
    7.42531255e-01, -1.66747506e+00,  7.42531255e-01,  6.80436429e-01 ,
   -3.04191736e+00, -4.22783385e+00,  1.50000000e+01, -1.50000000e+01 ,
    1.50000000e+01,  1.50000000e+01,]],

 [[ 8.74649714e-01,  8.81176607e-01, -9.64851511e-01, -1.66747506e+00 ,
    7.42531255e-01, -1.66747506e+00,  7.42531255e-01,  2.25649489e-01 ,
   -2.79986189e+00, -4.89804825e+00, -1.31465411e+01,  1.50000000e+01 ,
   -1.31465411e+01,  1.50000000e+01,],
  [ 9.18472986e-01,  7.58119240e-01, -1.04719755e+00, -2.02793684e+00 ,
    1.26513968e+00, -2.02793684e+00,  1.26513968e+00,  7.15846151e-01 ,
   -1.14690146e+00,  9.48676469e-01, -9.38761048e-01,  4.47318525e+00 ,
   -9.38761048e-01,  4.47318525e+00,],
  [ 9.68034385e-01,  7.27183936e-01, -9.09910648e-01, -1.91578437e+00 ,
    1.41108933e+00, -1.91578437e+00,  1.41108933e+00,  8.79376564e-01 ,
    6.18716551e-02,  3.42567748e+00,  4.24313297e+00,  6.20088488e-01 ,
    4.24313297e+00,  6.20088488e-01,],
  [ 1.04666883e+00,  7.48465189e-01, -5.76411748e-01, -1.47312449e+00 ,
    1.40670400e+00, -1.47312449e+00,  1.40670400e+00,  1.55185565e+00 ,
    4.40300855e-01,  6.59406264e+00,  8.90883855e+00, -4.14033333e-01 ,
    8.90883855e+00, -4.14033333e-01,],
  [ 1.15452732e+00,  7.81050304e-01, -1.40908840e-01, -8.63530934e-01 ,
    1.34570817e+00, -8.63530934e-01,  1.34570817e+00,  1.30465984e+00 ,
    5.01911511e-01,  4.87265695e+00,  7.27099551e+00, -1.40042409e+00 ,
    7.27099551e+00, -1.40042409e+00,],
  [ 1.20000000e+00,  8.00000000e-01,  1.61360323e-13, -6.43501110e-01 ,
    1.28700222e+00, -6.43501110e-01,  1.28700222e+00,  2.00000000e-01 ,
   -1.72519455e-13, -3.24280732e-14,  0.00000000e+00,  2.21451424e-13 ,
   -6.56060688e-14,  0.00000000e+00,]]]
).reshape(-1, 14)

n_mode = 3
N = 6
repeat = 2
LEN = n_mode * N
x_traj_jump_repeat = np.concatenate([x_traj_jump_repeat for _ in range(repeat)], axis=0)
x_traj_jump_repeat_dist = 1.2
for i in range(1, repeat):
  x_traj_jump_repeat[LEN * i: LEN*(i+1) , 0] += x_traj_jump_repeat_dist * i




In [ ]:
x_traj_jump_repeat_po = PiecewisePolynomial.ZeroOrderHold(np.linspace(0, 0.1 * x_traj_jump_repeat.shape[0], 
                                                            x_traj_jump_repeat.shape[0]),
                                                            x_traj_jump_repeat.T)
vis(x_traj_jump_repeat_po, 0.1 * x_traj_jump_repeat.shape[0], time_step=1)

In [ ]:
%tb
import importlib
import traj_optim
import fsm_utils

importlib.reload(traj_optim)
importlib.reload(fsm_utils)
from traj_optim import TrajectoryOptimizationSolution, read_csv
from fsm_utils import (
    LEFT_STANCE, RIGHT_STANCE, SPACE_STANCE, DOUBLE_SUPPORT
)
solver = TrajectoryOptimizationSolution(n_foot=6)
N = 6
mode_seqs = [DOUBLE_SUPPORT, SPACE_STANCE, DOUBLE_SUPPORT] #, LEFT_STANCE, RIGHT_STANCE]
repeat = 2
n_mode = len(mode_seqs)

#initial_guesses = x_traj_jump_repeat

#initial_state = np.array(
#    [0.00000000,0.80000000,0.00000000,
#              -0.64350111,1.28700222,-0.64350111,1.28700222,
#              0.20000000,0.00000000,0.00000000,0.00000000,
#              0.00000000,0.00000000,0.00000000]
#      )
#initial_guesses = np.concatenate([initial_state for _ in range(n_mode * N * repeat)]).reshape(n_mode * N * repeat, 14)
#initial_guesses[:, 0] = np.linspace(initial_state[0], destination, n_mode * N * repeat)

tf = 1.0 * repeat 
destination = 1.2 * repeat * 1.1

x_traj_jump_repeat[:, 0] = np.linspace(0, destination, x_traj_jump_repeat.shape[0])


x_traj_jump_2, u_traj_jump_2, prog = solver.solve(N, 
                                    mode_seqs, 
                                    repeat, 
                                    x_traj_jump_repeat,
                                    #initial_guesses,
                                     mu=5, 
                                     tf=tf, 
                                     destination=destination, 
                                     iters=20e4,
                                     test=False,
                                     final_hard_constraint=True,
                                     dest_cost=False
                                     )

In [ ]:
vis(x_traj_jump_2, tf, time_step=0.3)

## Unit Walking

In [ ]:
def cubtraj(solver, seqs, repeat, x_sol, u_sol, lambda_sol, timesteps):
    for m, fsm in enumerate(seqs * repeat):
            for i in range(N):
                J_i, _ = solver.CalculateContactJacobian(x_sol[m, i], fsm, False)
                xdot_sol[m, i] = solver.EvaluateDynamics(x_sol[m, i],
                                                        u_sol[m, i],
                                                        J_i, 
                                                        lambda_sol[m, i],
                                                        False)
        
    x_sol = x_sol.reshape(-1, solver.n_x)
    xdot_sol = xdot_sol.reshape(-1, solver.n_x)
    u_sol = u_sol.reshape(-1, solver.n_u)
    x_traj = PiecewisePolynomial.CubicHermite(timesteps, x_sol.T, xdot_sol.T)
    return x_traj

In [45]:
from pydrake.systems.pyplot_visualizer import PyPlotVisualizer
from pydrake.all import PiecewisePolynomial
from pydrake.geometry import MeshcatAnimation

x_sol = np.array([
  [ 0.00000000e+00,  8.00000000e-01, -1.74532925e-01, -3.49065850e-01,
    1.08269946e+00, -8.72664626e-01,  1.28269064e+00,  2.00000000e+00,
   -5.71508137e-15, -7.79373746e-15, -8.21144651e-19,  2.40148336e-18,
   -1.49291338e-13,  1.98306501e-13],
  [ 8.37034099e-03,  7.99925828e-01, -1.75257400e-01, -3.47384879e-01,
    1.47933616e+00, -8.58191193e-01,  1.28495518e+00,  2.09576631e+00,
    1.22841783e-01,  4.35549069e-02,  1.50000000e+01, -1.50000000e+01,
    2.72870110e+00, -1.26765222e-01],
  [ 3.71182625e-02,  8.01846282e-01, -1.75001097e-01,  8.69113800e-02,
    3.66531233e-01, -8.19849346e-01,  1.28065977e+00,  1.94430177e+00,
    7.34747863e-02, -5.09871323e-01,  1.50000000e+01, -1.50000000e+01,
    2.02341394e+00, -2.16546728e-01],
  [ 6.08878643e-02,  8.02682609e-01, -1.84223273e-01,  3.63501520e-02,
    1.19751250e+00, -7.97632924e-01,  1.27701790e+00,  2.05954448e+00,
    1.44397529e-01,  6.46797237e-02,  1.50000000e+01, -8.03350835e+00,
    2.95314470e+00, -6.57852798e-01],
  [ 9.04924929e-02,  8.04017918e-01, -1.76004709e-01,  5.12411299e-01,
   -1.07176121e-01, -7.48273232e-01,  1.26820816e+00,  1.99349570e+00,
    1.07820645e-01,  1.08073333e-01,  1.09379133e+01, -1.50000000e+01,
    2.95918558e+00, -7.79146322e-01],
  [ 1.13886976e-01,  8.06220669e-01, -1.81007748e-01,  2.94865813e-01,
    9.65015670e-01, -7.17842875e-01,  1.25474330e+00,  1.99704023e+00,
    1.54717137e-01,  3.15145320e-01,  6.24157942e+00, -8.59886907e+00,
    3.32461467e+00, -1.14013916e+00],
  [ 1.42839766e-01,  8.07761255e-01, -1.68499899e-01,  6.57482867e-01,
   -2.86519715e-01, -6.62097514e-01,  1.23883284e+00,  1.96959329e+00,
    1.82660076e-01,  4.78451114e-01,  9.21187498e-01, -2.95295205e+00,
    3.58899657e+00, -1.47782258e+00],
  [ 1.65473569e-01,  8.10863545e-01, -1.70742047e-01,  2.90952980e-01,
    9.60866204e-01, -6.26650393e-01,  1.21730743e+00,  1.79927480e+00,
    1.64639591e-01,  1.08495040e-01, -9.47592728e+00,  1.50000000e+01,
    3.00894884e+00, -1.52680676e+00],
  [ 1.92172392e-01,  8.12560780e-01, -1.62329592e-01,  4.69644503e-01,
   -1.95530027e-02, -5.75759225e-01,  1.19561742e+00,  1.81951034e+00,
    1.34888753e-01,  2.66125202e-01, -9.87412076e+00,  1.50000000e+01,
    3.22931250e+00, -1.65853299e+00],
  [ 2.12706127e-01,  8.13447693e-01, -1.67484967e-01,  3.84613583e-02,
    1.25591551e+00, -5.48089892e-01,  1.17816448e+00,  1.56046076e+00,
   -5.95297838e-02, -5.11996141e-01, -1.50000000e+01,  1.50000000e+01,
    1.81545158e+00, -9.52949231e-01],
  [ 2.34690784e-01,  8.12377056e-01, -1.72968336e-01, -1.02755552e-02,
    7.90775201e-01, -5.20702051e-01,  1.16445988e+00,  1.55322837e+00,
   -2.19688874e-01, -5.56110805e-01, -1.30422567e+01,  1.50000000e+01,
    1.58146057e+00, -5.35458272e-01],
  [ 2.54389714e-01,  8.08005740e-01, -1.80762361e-01, -3.07810833e-01,
    1.60021390e+00, -5.03309419e-01,  1.16237030e+00,  1.60605167e+00,
   -2.63490906e-01, -3.42117364e-01, -1.50000000e+01,  1.50000000e+01,
    1.88037025e+00, -5.68449075e-01],
  [ 2.77313450e-01,  8.05988111e-01, -1.82373803e-01, -4.14007920e-01,
    1.19997201e+00, -4.70172638e-01,  1.14685946e+00,  1.73633778e+00,
   -2.17483007e-01, -8.00362112e-02, -9.73640209e+00, -1.27614749e+01,
    2.49588796e+00, -1.04075148e+00],
  [ 3.00475749e-01,  8.01922890e-01, -1.82275170e-01, -6.31627931e-01,
    1.49702494e+00, -4.36869985e-01,  1.13589354e+00,  1.86520433e+00,
   -2.22566966e-01,  3.07801714e-01, -1.50000000e+01,  8.86723250e+00,
    3.16925517e+00, -1.35049927e+00],
  [ 3.26736690e-01,  8.00903200e-01, -1.75793288e-01, -7.62366097e-01,
    1.24831204e+00, -3.79139149e-01,  1.10214119e+00,  1.99713427e+00,
   -5.25169006e-02,  3.65086113e-01, -9.25374279e+00, -1.50000000e+01,
    4.90712406e+00, -3.23573096e+00],
  [ 3.52842475e-01,  8.00000000e-01, -1.74532925e-01, -8.72664626e-01,
    1.28269064e+00, -3.49065850e-01,  1.08269946e+00,  2.00000000e+00,
   -1.28679629e-13,  1.88044157e-13, -7.40838872e-18, -2.75686821e-18,
   -1.98748591e-14,  1.06564554e-13]
])


u_sol = np.array(  
[[[ 279.48167308,   -5.72417158,  260.23612204,   -1.32538728], 
  [-297.23404144,  375.48654526, -256.0991747 ,  -71.96528483], 
  [ -63.40754462,  183.48232989,  360.84650747,  -94.492698  ], 
  [-500.        ,    6.72064144, -265.09422525,    2.9710277 ], 
  [ -82.08244707,    2.0677216 ,  159.36918762,   -2.77679019], 
  [ 182.19728604,  354.80321909,  -79.05863049, -396.70119994], 
  [-248.25060148,  -13.54134801,   31.97960992,  -14.23851461], 
  [ 500.        , -410.98700358,   58.32064619, -400.        ],]]
)

lambda_sol = np.array(
[[[   0.        ,    0.,            0.        ],
  [ 452.26885609,    0.,          904.53771219],
  [ 282.54041222,    0.,          634.86245717],
  [   0.        ,    0.,            0.        ],
  [  -3.46936787,    0.,            6.93873575],
  [ 947.0275757 ,    0.,         1894.05515141],
  [ -13.89568196,    0.,           27.79136391],
  [-171.73301856,    0.,          886.04639429],]]
)

x_sol_flip = x_sol.copy()
x_sol_flip[:, 5:7] = x_sol[:, 3:5]
x_sol_flip[:, 12:] = x_sol[:, 10:12]
x_sol_flip[:, 3:5] = x_sol[:, 5:7]
x_sol_flip[:, 10:12] = x_sol[:, 12:]


steps = 8
x_offset = 3.52842475e-01
x_sol_flip[:, 0] += x_offset
x_repeated = np.vstack((x_sol, x_sol_flip))
for i in range(steps-2):
    if i%2 == 0:
        x_sol[:, 0] += x_offset*2
        x_repeated = np.vstack((x_repeated, x_sol))
    else:
        x_sol_flip[:, 0] += x_offset*2
        x_repeated = np.vstack((x_repeated, x_sol_flip))
    
x_sol_traj = PiecewisePolynomial.ZeroOrderHold(np.linspace(0, 3, x_repeated.shape[0]), 
                                   x_repeated.T
                                           )

vis(x_sol_traj, 3, time_step=0.3)

## Walking

In [5]:
#%tb

import importlib
import traj_optim
import fsm_utils

importlib.reload(traj_optim)
importlib.reload(fsm_utils)
from traj_optim import TrajectoryOptimizationSolution, read_csv
from fsm_utils import (
    LEFT_STANCE, RIGHT_STANCE, SPACE_STANCE, DOUBLE_SUPPORT
)


solver = TrajectoryOptimizationSolution(n_foot=3)
N = 8
mode_seqs = [RIGHT_STANCE, LEFT_STANCE]
repeat = 4
n_mode = len(mode_seqs)
destination = 0.35 * 8

xl = np.array(
        [-1.42628695e-01, 7.91008947e-01, -1.42999388e-01,
          -7.89749727e-01, 1.31650877e+00, -2.43590062e-01, 1.04394874e+00, 
          0, 0, 0, 
          0, 0, 0, 0]
      )
xr = np.array(
          [-1.42628695e-01, 7.91008947e-01, -1.42999388e-01,
         -2.43590062e-01, 1.04394874e+00,  -7.89749727e-01, 1.31650877e+00, 
          0, 0, 0, 
          0, 0, 0, 0]
)
initial_guesses = np.concatenate([xl if (i // N) % n_mode == 0 else xr 
                                  for i in range(n_mode * N * repeat)]).reshape(n_mode * N * repeat, 14)
initial_guesses[:, 0] = np.linspace(xl[0], destination, n_mode * N * repeat)

## Deprecated Above

xr = np.array(
[[[ 2.57978226e-13,  8.00000000e-01 ,-1.74532925e-01, -3.49065850e-01,
    1.08269946e+00, -8.72664626e-01 , 1.28269064e+00,  2.00000000e+00,
    9.39283730e-14, -8.66523524e-14 ,-2.22696773e-13, -8.97949647e-17,
   -2.00745126e-14,  1.42708062e-13],                                
  [ 6.31372136e-02,  7.95209029e-01 ,-1.48793403e-01, -9.02332565e-02,
    2.18779134e+00, -7.87504013e-01 , 1.25056405e+00,  2.41077310e+00,
   -6.15594322e-02,  1.52181396e+00 , 1.40881243e+01, -1.50000000e+01,
    6.18765772e+00, -3.35198754e+00],                                 
  [ 1.17984652e-01,  8.16533973e-01 ,-1.11236606e-01,  6.05293936e-01,
   -6.66837235e-01, -6.20084914e-01 , 1.21639092e+00,  2.32339822e+00,
   -1.20019802e-01,  1.90733836e+00 , 9.92429771e+00, -6.21096464e-01,
    4.93323561e+00, -3.88577118e-01],                                 
  [ 1.74967521e-01,  8.13729077e-01 ,-7.77727397e-02,  5.27219466e-01,
    2.16768563e+00, -5.08896419e-01 , 1.19963188e+00,  1.59921014e+00,
   -2.32206195e-01, -2.13549514e-01 , 4.03286558e+00,  1.50000000e+01,
    1.81371265e+00, -1.40257220e-01],                                 
  [ 2.13557065e-01,  8.09771779e-01 ,-1.22696199e-01,  3.56032290e-01,
    7.84341177e-01, -5.00853104e-01 , 1.18714573e+00,  1.05128180e+00,
   -2.18698343e-01, -2.63655148e+00 ,-1.50000000e+01, -1.50000000e+01,
   -1.32146027e+00, -3.95932965e-02],                                
  [ 2.43819596e-01,  8.01225854e-01 ,-1.87658377e-01, -3.61408802e-01,
    1.74938579e+00, -5.30147217e-01 , 1.19174995e+00,  1.51849462e+00,
   -1.83285877e-01, -7.79394735e-01 ,-1.50000000e+01,  9.09746418e+00,
    1.41433502e+00, -7.24219044e-01],                                 
  [ 2.99380207e-01,  8.02132986e-01 ,-1.73383809e-01, -5.04102832e-01,
    1.18275196e+00, -4.05476482e-01 , 1.11930249e+00,  1.99335348e+00,
    3.12577908e-02,  5.32129489e-01 ,-8.86385954e+00, -1.50000000e+01,
    5.18339143e+00, -3.35198917e+00],                                 
  [ 3.52842475e-01,  8.00000000e-01 ,-1.74532925e-01, -8.72664626e-01,
    1.28269064e+00, -3.49065850e-01 , 1.08269946e+00,  2.00000000e+00,
    3.88483585e-14,  7.99961634e-14 ,-2.57457468e-13,  5.34875567e-14,
   -1.43408199e-13,  7.33050277e-14]]]
)

xl = xr.copy()
xl[0, :, 0] += xr[0, -1, 0]
xl[0, :, 3:5] = xl[0, :, 5:7]
xl[0, :, 5:7] = xr[0, :, 3:5]

initial_guesses = np.concatenate([xr, xl, xr, xl, xr, xl, xr, xl], axis=1)
for i in range(1, repeat * n_mode):
  initial_guesses[0, i*N: (i+1)*N, 0] += i * (initial_guesses[0, N-1, 0] - initial_guesses[0, 0, 0])

initial_guesses = initial_guesses.reshape(n_mode * N * repeat, 14)
initial_guesses[:, 0] = np.linspace(xl[0, 0, 0], destination, n_mode * N * repeat)

uguessr = np.array(
[[[ 279.48167308,   -5.72417158,  260.23612204,   -1.32538728], 
  [-297.23404144,  375.48654526, -256.0991747 ,  -71.96528483], 
  [ -63.40754462,  183.48232989,  360.84650747,  -94.492698  ], 
  [-500.        ,    6.72064144, -265.09422525,    2.9710277 ], 
  [ -82.08244707,    2.0677216 ,  159.36918762,   -2.77679019], 
  [ 182.19728604,  354.80321909,  -79.05863049, -396.70119994], 
  [-248.25060148,  -13.54134801,   31.97960992,  -14.23851461], 
  [ 500.        , -410.98700358,   58.32064619, -400.        ],]]
)
uguessl = uguessr.copy()
uguessl[0, :, 0:2] = uguessr[0, :, 2:4]
uguessl[0, :, 2:4] = uguessr[0, :, 0:2]
uguesses = np.concatenate([uguessl, uguessr, uguessl, uguessr, uguessl, uguessr, uguessl, uguessr], axis=1).reshape(n_mode * N * repeat, 4)

In [6]:
#initial_guesses
LEN = n_mode * N

x_traj_g= PiecewisePolynomial.ZeroOrderHold(np.linspace(0, 0.1 * initial_guesses.shape[0], initial_guesses.shape[0]), initial_guesses.T)
vis(x_traj_g, 0.1 * initial_guesses.shape[0], time_step=0.4)

In [7]:
tf = 0.4 * 4# just now is tf = 2
destination = 0.35 * 8

x_traj_walk, u_traj_walk, prog = solver.solve(N, 
                                    mode_seqs, 
                                    repeat, 
                                    initial_guesses,
                                     mu=1, 
                                     tf=tf, 
                                     destination=destination, 
                                     iters=20e4,
                                     test=False,
                                     uguess=None)

Whole x shape:  (8, 8, 14)
Timesteps:  [1.00000000e-03 2.94285714e-02 5.78571429e-02 8.62857143e-02
 1.14714286e-01 1.43142857e-01 1.71571429e-01 2.00000000e-01
 2.01000000e-01 2.29428571e-01 2.57857143e-01 2.86285714e-01
 3.14714286e-01 3.43142857e-01 3.71571429e-01 4.00000000e-01
 4.01000000e-01 4.29428571e-01 4.57857143e-01 4.86285714e-01
 5.14714286e-01 5.43142857e-01 5.71571429e-01 6.00000000e-01
 6.01000000e-01 6.29428571e-01 6.57857143e-01 6.86285714e-01
 7.14714286e-01 7.43142857e-01 7.71571429e-01 8.00000000e-01
 8.01000000e-01 8.29428571e-01 8.57857143e-01 8.86285714e-01
 9.14714286e-01 9.43142857e-01 9.71571429e-01 1.00000000e+00
 1.00100000e+00 1.02942857e+00 1.05785714e+00 1.08628571e+00
 1.11471429e+00 1.14314286e+00 1.17157143e+00 1.20000000e+00
 1.20100000e+00 1.22942857e+00 1.25785714e+00 1.28628571e+00
 1.31471429e+00 1.34314286e+00 1.37157143e+00 1.40000000e+00
 1.40100000e+00 1.42942857e+00 1.45785714e+00 1.48628571e+00
 1.51471429e+00 1.54314286e+00 1.57157143e+00 

In [10]:
vis(x_traj_walk, tf, 0.1)

In [15]:
# RUNNING?
solver = TrajectoryOptimizationSolution(n_foot=3)
N = 8
mode_seqs = [RIGHT_STANCE, LEFT_STANCE]
repeat = 4
n_mode = len(mode_seqs)
tf = 0.4 * 3# just now is tf = 2
destination = 0.35 * 8.5

xr = np.array(
[[[ 2.57978226e-13,  8.00000000e-01 ,-1.74532925e-01, -3.49065850e-01,
    1.08269946e+00, -8.72664626e-01 , 1.28269064e+00,  2.00000000e+00,
    9.39283730e-14, -8.66523524e-14 ,-2.22696773e-13, -8.97949647e-17,
   -2.00745126e-14,  1.42708062e-13],                                
  [ 6.31372136e-02,  7.95209029e-01 ,-1.48793403e-01, -9.02332565e-02,
    2.18779134e+00, -7.87504013e-01 , 1.25056405e+00,  2.41077310e+00,
   -6.15594322e-02,  1.52181396e+00 , 1.40881243e+01, -1.50000000e+01,
    6.18765772e+00, -3.35198754e+00],                                 
  [ 1.17984652e-01,  8.16533973e-01 ,-1.11236606e-01,  6.05293936e-01,
   -6.66837235e-01, -6.20084914e-01 , 1.21639092e+00,  2.32339822e+00,
   -1.20019802e-01,  1.90733836e+00 , 9.92429771e+00, -6.21096464e-01,
    4.93323561e+00, -3.88577118e-01],                                 
  [ 1.74967521e-01,  8.13729077e-01 ,-7.77727397e-02,  5.27219466e-01,
    2.16768563e+00, -5.08896419e-01 , 1.19963188e+00,  1.59921014e+00,
   -2.32206195e-01, -2.13549514e-01 , 4.03286558e+00,  1.50000000e+01,
    1.81371265e+00, -1.40257220e-01],                                 
  [ 2.13557065e-01,  8.09771779e-01 ,-1.22696199e-01,  3.56032290e-01,
    7.84341177e-01, -5.00853104e-01 , 1.18714573e+00,  1.05128180e+00,
   -2.18698343e-01, -2.63655148e+00 ,-1.50000000e+01, -1.50000000e+01,
   -1.32146027e+00, -3.95932965e-02],                                
  [ 2.43819596e-01,  8.01225854e-01 ,-1.87658377e-01, -3.61408802e-01,
    1.74938579e+00, -5.30147217e-01 , 1.19174995e+00,  1.51849462e+00,
   -1.83285877e-01, -7.79394735e-01 ,-1.50000000e+01,  9.09746418e+00,
    1.41433502e+00, -7.24219044e-01],                                 
  [ 2.99380207e-01,  8.02132986e-01 ,-1.73383809e-01, -5.04102832e-01,
    1.18275196e+00, -4.05476482e-01 , 1.11930249e+00,  1.99335348e+00,
    3.12577908e-02,  5.32129489e-01 ,-8.86385954e+00, -1.50000000e+01,
    5.18339143e+00, -3.35198917e+00],                                 
  [ 3.52842475e-01,  8.00000000e-01 ,-1.74532925e-01, -8.72664626e-01,
    1.28269064e+00, -3.49065850e-01 , 1.08269946e+00,  2.00000000e+00,
    3.88483585e-14,  7.99961634e-14 ,-2.57457468e-13,  5.34875567e-14,
   -1.43408199e-13,  7.33050277e-14]]]
)

xl = xr.copy()
xl[0, :, 0] += xr[0, -1, 0]
xl[0, :, 3:5] = xl[0, :, 5:7]
xl[0, :, 5:7] = xr[0, :, 3:5]

initial_guesses2 = np.concatenate([xr, xl, xr, xl, xr, xl, xr, xl], axis=1)
for i in range(1, repeat * n_mode):
  initial_guesses2[0, i*N: (i+1)*N, 0] += i * (initial_guesses2[0, N-1, 0] - initial_guesses2[0, 0, 0])

initial_guesses2 = initial_guesses2.reshape(n_mode * N * repeat, 14)
initial_guesses2[:, 0] = np.linspace(xr[0, 0, 0], destination, n_mode * N * repeat)

x_traj_runnn, u_traj_runnn, prog = solver.solve(N, 
                                    mode_seqs, 
                                    repeat, 
                                    initial_guesses2,
                                     mu=1, 
                                     tf=tf, 
                                     destination=destination, 
                                     iters=10e4,
                                     test=False,
                                     uguess=None,
                                     dest_cost=True
                                     )

Whole x shape:  (8, 8, 14)
Timesteps:  [1.00000000e-03 2.22857143e-02 4.35714286e-02 6.48571429e-02
 8.61428571e-02 1.07428571e-01 1.28714286e-01 1.50000000e-01
 1.51000000e-01 1.72285714e-01 1.93571429e-01 2.14857143e-01
 2.36142857e-01 2.57428571e-01 2.78714286e-01 3.00000000e-01
 3.01000000e-01 3.22285714e-01 3.43571429e-01 3.64857143e-01
 3.86142857e-01 4.07428571e-01 4.28714286e-01 4.50000000e-01
 4.51000000e-01 4.72285714e-01 4.93571429e-01 5.14857143e-01
 5.36142857e-01 5.57428571e-01 5.78714286e-01 6.00000000e-01
 6.01000000e-01 6.22285714e-01 6.43571429e-01 6.64857143e-01
 6.86142857e-01 7.07428571e-01 7.28714286e-01 7.50000000e-01
 7.51000000e-01 7.72285714e-01 7.93571429e-01 8.14857143e-01
 8.36142857e-01 8.57428571e-01 8.78714286e-01 9.00000000e-01
 9.01000000e-01 9.22285714e-01 9.43571429e-01 9.64857143e-01
 9.86142857e-01 1.00742857e+00 1.02871429e+00 1.05000000e+00
 1.05100000e+00 1.07228571e+00 1.09357143e+00 1.11485714e+00
 1.13614286e+00 1.15742857e+00 1.17871429e+00 

In [40]:
vis(x_traj_runnn, 1.2, 0.1)

## Running

In [102]:
#%tb

import importlib
import traj_optim
import fsm_utils

importlib.reload(traj_optim)
importlib.reload(fsm_utils)
from traj_optim import TrajectoryOptimizationSolution, read_csv
from fsm_utils import (
    LEFT_STANCE, RIGHT_STANCE, SPACE_STANCE, DOUBLE_SUPPORT
)


solver = TrajectoryOptimizationSolution(n_foot=6)
N = 4
mode_seqs = [
    RIGHT_STANCE, SPACE_STANCE, LEFT_STANCE, SPACE_STANCE,
    RIGHT_STANCE, SPACE_STANCE, LEFT_STANCE, SPACE_STANCE,
    RIGHT_STANCE, SPACE_STANCE, LEFT_STANCE, SPACE_STANCE,
    RIGHT_STANCE, SPACE_STANCE, LEFT_STANCE, DOUBLE_SUPPORT
             ] #, LEFT_STANCE, RIGHT_STANCE]
repeat = 1
n_mode = len(mode_seqs)
#initial_states = read_csv("q&v.csv", N * repeat * len(mode_seqs))

initial_state = np.array(
    [0.00000000,0.80000000,0.00000000,
              -0.64350111,1.28700222,-0.64350111,1.28700222,
              0.20000000,0.00000000,0.00000000,0.00000000,
              0.00000000,0.00000000,0.00000000]
      )
initial_guesses = np.concatenate([initial_state for _ in range(n_mode * N * repeat)]).reshape(n_mode * N * repeat, 14)
initial_guesses[:, 0] = np.linspace(initial_state[0], destination, n_mode * N * repeat)

xr = np.array(
[[[ 2.57978226e-13,  8.00000000e-01 ,-1.74532925e-01, -3.49065850e-01,
    1.08269946e+00, -8.72664626e-01 , 1.28269064e+00,  2.00000000e+00,
    9.39283730e-14, -8.66523524e-14 ,-2.22696773e-13, -8.97949647e-17,
   -2.00745126e-14,  1.42708062e-13],                                
  [ 6.31372136e-02,  7.95209029e-01 ,-1.48793403e-01, -9.02332565e-02,
    2.18779134e+00, -7.87504013e-01 , 1.25056405e+00,  2.41077310e+00,
   -6.15594322e-02,  1.52181396e+00 , 1.40881243e+01, -1.50000000e+01,
    6.18765772e+00, -3.35198754e+00],                                 
  [ 1.17984652e-01,  8.16533973e-01 ,-1.11236606e-01,  6.05293936e-01,
   -6.66837235e-01, -6.20084914e-01 , 1.21639092e+00,  2.32339822e+00,
   -1.20019802e-01,  1.90733836e+00 , 9.92429771e+00, -6.21096464e-01,
    4.93323561e+00, -3.88577118e-01],                                 
  [ 1.74967521e-01,  8.13729077e-01 ,-7.77727397e-02,  5.27219466e-01,
    2.16768563e+00, -5.08896419e-01 , 1.19963188e+00,  1.59921014e+00,
   -2.32206195e-01, -2.13549514e-01 , 4.03286558e+00,  1.50000000e+01,
    1.81371265e+00, -1.40257220e-01],                                 
  [ 2.13557065e-01,  8.09771779e-01 ,-1.22696199e-01,  3.56032290e-01,
    7.84341177e-01, -5.00853104e-01 , 1.18714573e+00,  1.05128180e+00,
   -2.18698343e-01, -2.63655148e+00 ,-1.50000000e+01, -1.50000000e+01,
   -1.32146027e+00, -3.95932965e-02],                                
  [ 2.43819596e-01,  8.01225854e-01 ,-1.87658377e-01, -3.61408802e-01,
    1.74938579e+00, -5.30147217e-01 , 1.19174995e+00,  1.51849462e+00,
   -1.83285877e-01, -7.79394735e-01 ,-1.50000000e+01,  9.09746418e+00,
    1.41433502e+00, -7.24219044e-01],                                 
  [ 2.99380207e-01,  8.02132986e-01 ,-1.73383809e-01, -5.04102832e-01,
    1.18275196e+00, -4.05476482e-01 , 1.11930249e+00,  1.99335348e+00,
    3.12577908e-02,  5.32129489e-01 ,-8.86385954e+00, -1.50000000e+01,
    5.18339143e+00, -3.35198917e+00],                                 
  [ 3.52842475e-01,  8.00000000e-01 ,-1.74532925e-01, -8.72664626e-01,
    1.28269064e+00, -3.49065850e-01 , 1.08269946e+00,  2.00000000e+00,
    3.88483585e-14,  7.99961634e-14 ,-2.57457468e-13,  5.34875567e-14,
   -1.43408199e-13,  7.33050277e-14]]]
)

xl = xr.copy()
xl[0, :, 0] += xr[0, -1, 0]
xl[0, :, 3:5] = xl[0, :, 5:7]
xl[0, :, 5:7] = xr[0, :, 3:5]

initial_guesses2 = np.concatenate([xr, xl, xr, xl, xr, xl, xr, xl], axis=1)
for i in range(1, repeat * n_mode):
  initial_guesses2[0, i*N: (i+1)*N, 0] += i * (initial_guesses2[0, N-1, 0] - initial_guesses2[0, 0, 0])

initial_guesses2 = initial_guesses2.reshape(n_mode * N * repeat, 14)
initial_guesses2[:, 0] = np.linspace(xr[0, 0, 0], destination, n_mode * N * repeat)


tf = 0.6 # just now is tf = 2
destination = .35 * 8.5

x_traj_run, u_traj_run, prog = solver.solve(N, 
                                    mode_seqs, 
                                    repeat, 
                                    initial_guesses2,
                                     mu=1, 
                                     tf=tf, 
                                     destination=destination, 
                                     iters=10e4,
                                     test=False,
                                     dest_cost=True
                                     )

Whole x shape:  (16, 4, 14)
Timesteps:  [0.001      0.01316667 0.02533333 0.0375     0.0385     0.05066667
 0.06283333 0.075      0.076      0.08816667 0.10033333 0.1125
 0.1135     0.12566667 0.13783333 0.15       0.151      0.16316667
 0.17533333 0.1875     0.1885     0.20066667 0.21283333 0.225
 0.226      0.23816667 0.25033333 0.2625     0.2635     0.27566667
 0.28783333 0.3        0.301      0.31316667 0.32533333 0.3375
 0.3385     0.35066667 0.36283333 0.375      0.376      0.38816667
 0.40033333 0.4125     0.4135     0.42566667 0.43783333 0.45
 0.451      0.46316667 0.47533333 0.4875     0.4885     0.50066667
 0.51283333 0.525      0.526      0.53816667 0.55033333 0.5625
 0.5635     0.57566667 0.58783333 0.6       ]
Destination: 2.9749999999999996m
mu = 1
Testing destination cost
Testing  100000.0  iterations
Time taken: 797.7300104590249 seconds
Check whether the friction cone constraint holds?
Friction Cone Constraint Not Hold
lambda_c_m15_2: [-4.80974553e-08  0.00000000e+00  

In [104]:
vis(x_traj_run, tf, 0.1)

In [93]:
import importlib
import traj_optim
import fsm_utils

importlib.reload(fsm_utils)
importlib.reload(traj_optim)
from traj_optim import TrajectoryOptimizationSolution, read_csv
from fsm_utils import (
    LEFT_STANCE, RIGHT_STANCE, SPACE_STANCE, DOUBLE_SUPPORT
)


run_initial_unit = np.array(
[[[ 3.79730938e-13,  8.00000000e-01, -1.74532925e-01,
         -3.49065850e-01,  1.08269946e+00, -8.72664626e-01,
          1.28269064e+00,  2.00000000e+00,  2.13654029e-13,
         -1.79647460e-13, -3.43242246e-13,  1.02027503e-13,
         -1.54824568e-14,  3.06615883e-13],
        [ 3.28533683e-02,  7.98726574e-01, -1.73309705e-01,
         -3.77225231e-01,  1.23471911e+00, -8.54292646e-01,
          1.28949615e+00,  2.06118581e+00, -1.53907434e-01,
          2.49252783e-01, -2.14877028e-01,  1.17993951e+01,
          2.43930433e+00,  7.60350878e-01],
        [ 6.74485194e-02,  7.95038321e-01, -1.66155241e-01,
         -3.29963875e-01,  1.40550448e+00, -8.10753747e-01,
          1.30338677e+00,  2.15074061e+00, -3.04690124e-01,
          5.51548282e-01,  4.57235395e+00,  1.11232896e+01,
          2.78210037e+00,  9.54430917e-01],
        [ 1.02050023e-01,  7.88774612e-01, -1.58942724e-01,
         -2.76879467e-01,  1.60030993e+00, -7.68283286e-01,
          1.32062974e+00,  2.04457494e+00, -4.54612702e-01,
          1.82019021e-01,  1.30447909e-01,  1.12026023e+01,
          2.22620023e+00,  1.14098220e+00]],

       [[ 1.02050023e-01,  7.88794581e-01, -1.58942724e-01,
         -2.76938056e-01,  1.60030993e+00, -7.68283286e-01,
          1.32062974e+00,  4.00872423e+00,  6.47005442e-03,
         -2.31925385e-01, -7.36706411e+00,  1.21266814e+01,
          9.30978513e+00,  1.46469050e+01],
        [ 1.66447788e-01,  7.86970119e-01, -1.67067592e-01,
         -4.38481924e-01,  1.70685296e+00, -6.34362498e-01,
          1.81053942e+00,  3.92283560e+00, -1.87333325e-01,
         -5.98827832e-01, -1.07084772e+01,  1.14481164e+01,
          1.08632782e+01,  1.49958803e+01],
        [ 2.30218422e-01,  7.83269864e-01, -1.78501027e-01,
         -6.13052373e-01,  1.83883920e+00, -4.58383154e-01,
          1.68277280e+00,  3.83543638e+00, -2.59260518e-01,
         -9.75930942e-01, -1.14531183e+01, -1.46574748e+01,
          8.69305769e+00, -1.31358798e+01],
        [ 2.91590972e-01,  7.77872171e-01, -1.99358616e-01,
         -7.84018052e-01,  1.35017147e+00, -3.54883289e-01,
          1.55131320e+00,  3.72295802e+00, -4.60475687e-01,
         -1.42090598e+00, -1.17657658e+01, -1.27254662e+01,
          5.40936697e+00, -1.08307421e+01]],

       [[ 2.91590972e-01,  7.77872076e-01, -1.99358616e-01,
         -7.84018052e-01,  1.35017147e+00, -3.54883289e-01,
          1.55131320e+00,  3.81996667e+00, -4.78766308e-01,
         -1.27649108e+00,  3.42685621e+00,  4.61556801e-01,
          5.81845890e+00, -1.09658919e+01],
        [ 3.53932535e-01,  7.68574302e-01, -2.20086772e-01,
         -7.25974074e-01,  1.35456507e+00, -2.74141435e-01,
          1.44690465e+00,  3.80681045e+00, -6.65719028e-01,
         -1.27487103e+00,  3.64899165e+00,  9.68696414e-02,
          3.65012540e+00,  8.81573825e-01],
        [ 4.15867987e-01,  7.56143947e-01, -2.41341472e-01,
         -6.65234258e-01,  1.35329320e+00, -2.31533342e-01,
          1.58336914e+00,  3.77945835e+00, -8.50084341e-01,
         -1.33474670e+00,  3.79324010e+00, -2.74566680e-01,
          2.15871807e+00,  1.33889074e+01],
        [ 4.77538038e-01,  7.40933632e-01, -2.63515669e-01,
         -6.01503620e-01,  1.34499927e+00, -1.88936365e-01,
          1.81813903e+00,  3.77779882e+00, -1.01088613e+00,
         -1.36364826e+00,  4.03878679e+00, -7.55687030e-01,
          3.28165422e+00,  1.38103651e+01]],

       [[ 4.77538038e-01,  7.40933632e-01, -2.63515669e-01,
         -6.01503620e-01,  1.34499927e+00, -1.88936365e-01,
          1.81813903e+00,  7.32957556e+00,  2.04627130e-01,
          6.27166078e-01,  6.17835070e+00,  1.49421668e+01,
         -1.32966853e+01, -1.57176678e+00],
        [ 5.98193595e-01,  7.42187629e-01, -2.49536073e-01,
         -4.41617012e-01,  1.66526137e+00, -4.06772666e-01,
          1.84158173e+00,  7.40521754e+00, -3.02029066e-02,
          9.16356063e-01,  1.28625460e+01,  1.43944973e+01,
         -1.40618413e+01,  4.88467592e+00],
        [ 7.18886371e-01,  7.40412496e-01, -2.36072368e-01,
         -2.22973662e-01,  1.73940691e+00, -6.48710749e-01,
          1.83839988e+00,  7.38062218e+00, -1.75125885e-01,
          7.48945923e-01,  1.33351279e+01, -1.45966592e+00,
         -1.43718016e+01, -1.40867193e+01],
        [ 8.39340540e-01,  7.36161710e-01, -2.24720803e-01,
         -1.06356517e-02,  1.68658078e+00, -8.48463122e-01,
          1.47582001e+00,  7.36684540e+00, -3.87020597e-01,
          6.52333724e-01,  1.26872985e+01, -4.47689511e+00,
         -1.20152991e+01, -1.38531007e+01]]]
)

tf = 0.8 # just now is tf = 2
destination = 0.35 * 8.9
unit_repeat = 5
repeat = 1
N = 4
mode_seqs = [
    RIGHT_STANCE, SPACE_STANCE, LEFT_STANCE, SPACE_STANCE,
    RIGHT_STANCE, SPACE_STANCE, LEFT_STANCE, SPACE_STANCE,
    RIGHT_STANCE, SPACE_STANCE, LEFT_STANCE, SPACE_STANCE,
    RIGHT_STANCE, SPACE_STANCE, LEFT_STANCE, DOUBLE_SUPPORT
             ] 
n_mode = len(mode_seqs)

run_initial = np.concatenate([run_initial_unit for _ in range(unit_repeat)], axis=0).reshape(n_mode * repeat, N, 14)
for i in range(1, unit_repeat):
  run_initial[i*N: (i+1)*N, :, 0] += i * (run_initial[3, -1, 0] - run_initial[0, 0, 0])

run_initial = run_initial.reshape(n_mode * repeat * N, 14)

In [81]:
x_traj_run_g= PiecewisePolynomial.ZeroOrderHold(
                    np.linspace(0, 0.1 * run_initial.shape[0], run_initial.shape[0]), 
                    run_initial.T
                    )
vis(x_traj_run_g, 0.1 * run_initial.shape[0], time_step=1.8)

In [94]:
solver = TrajectoryOptimizationSolution(n_foot=6)


x_traj_run_2, u_traj_run_2, prog = solver.solve(N, 
                                    mode_seqs, 
                                    repeat, 
                                    run_initial,
                                     mu=1, 
                                     tf=tf, 
                                     destination=destination, 
                                     iters=9e4,
                                     test=False,
                                     dest_cost=True
                                     )

Whole x shape:  (20, 4, 14)
Timesteps:  [0.001 0.014 0.027 0.04  0.041 0.054 0.067 0.08  0.081 0.094 0.107 0.12
 0.121 0.134 0.147 0.16  0.161 0.174 0.187 0.2   0.201 0.214 0.227 0.24
 0.241 0.254 0.267 0.28  0.281 0.294 0.307 0.32  0.321 0.334 0.347 0.36
 0.361 0.374 0.387 0.4   0.401 0.414 0.427 0.44  0.441 0.454 0.467 0.48
 0.481 0.494 0.507 0.52  0.521 0.534 0.547 0.56  0.561 0.574 0.587 0.6
 0.601 0.614 0.627 0.64  0.641 0.654 0.667 0.68  0.681 0.694 0.707 0.72
 0.721 0.734 0.747 0.76  0.761 0.774 0.787 0.8  ]
Destination: 3.1m
mu = 1
Testing destination cost
Testing  90000.0  iterations
Time taken: 114.59588400000939 seconds
Check whether the friction cone constraint holds?
Friction Cone Constraint Not Hold
lambda_c_m6_0: [-4.27219986e-16  0.00000000e+00 -2.08479042e-13  0.00000000e+00
  0.00000000e+00  0.00000000e+00]
Check contact point velocity and acceleration be 0
Contact point velocity and acceleration be 0 Not Hold
h_m0_0: [<AutoDiffXd 0.0 nderiv=0> <AutoDiffXd 0.0 nderiv=

In [105]:
vis(x_traj_run_2, tf, 0.1)

## Backflipping

In [133]:
import importlib
import traj_optim
import fsm_utils

importlib.reload(traj_optim)
importlib.reload(fsm_utils)
from traj_optim import TrajectoryOptimizationSolution, read_csv
from fsm_utils import (
    LEFT_STANCE, RIGHT_STANCE, SPACE_STANCE, DOUBLE_SUPPORT, FLIP
)


run_initial_unit = np.array(
[[[ 3.79730938e-13,  8.00000000e-01, -1.74532925e-01,
         -3.49065850e-01,  1.08269946e+00, -8.72664626e-01,
          1.28269064e+00,  2.00000000e+00,  2.13654029e-13,
         -1.79647460e-13, -3.43242246e-13,  1.02027503e-13,
         -1.54824568e-14,  3.06615883e-13],
        [ 3.28533683e-02,  7.98726574e-01, -1.73309705e-01,
         -3.77225231e-01,  1.23471911e+00, -8.54292646e-01,
          1.28949615e+00,  2.06118581e+00, -1.53907434e-01,
          2.49252783e-01, -2.14877028e-01,  1.17993951e+01,
          2.43930433e+00,  7.60350878e-01],
        [ 6.74485194e-02,  7.95038321e-01, -1.66155241e-01,
         -3.29963875e-01,  1.40550448e+00, -8.10753747e-01,
          1.30338677e+00,  2.15074061e+00, -3.04690124e-01,
          5.51548282e-01,  4.57235395e+00,  1.11232896e+01,
          2.78210037e+00,  9.54430917e-01],
        [ 1.02050023e-01,  7.88774612e-01, -1.58942724e-01,
         -2.76879467e-01,  1.60030993e+00, -7.68283286e-01,
          1.32062974e+00,  2.04457494e+00, -4.54612702e-01,
          1.82019021e-01,  1.30447909e-01,  1.12026023e+01,
          2.22620023e+00,  1.14098220e+00]],

       [[ 1.02050023e-01,  7.88794581e-01, -1.58942724e-01,
         -2.76938056e-01,  1.60030993e+00, -7.68283286e-01,
          1.32062974e+00,  4.00872423e+00,  6.47005442e-03,
         -2.31925385e-01, -7.36706411e+00,  1.21266814e+01,
          9.30978513e+00,  1.46469050e+01],
        [ 1.66447788e-01,  7.86970119e-01, -1.67067592e-01,
         -4.38481924e-01,  1.70685296e+00, -6.34362498e-01,
          1.81053942e+00,  3.92283560e+00, -1.87333325e-01,
         -5.98827832e-01, -1.07084772e+01,  1.14481164e+01,
          1.08632782e+01,  1.49958803e+01],
        [ 2.30218422e-01,  7.83269864e-01, -1.78501027e-01,
         -6.13052373e-01,  1.83883920e+00, -4.58383154e-01,
          1.68277280e+00,  3.83543638e+00, -2.59260518e-01,
         -9.75930942e-01, -1.14531183e+01, -1.46574748e+01,
          8.69305769e+00, -1.31358798e+01],
        [ 2.91590972e-01,  7.77872171e-01, -1.99358616e-01,
         -7.84018052e-01,  1.35017147e+00, -3.54883289e-01,
          1.55131320e+00,  3.72295802e+00, -4.60475687e-01,
         -1.42090598e+00, -1.17657658e+01, -1.27254662e+01,
          5.40936697e+00, -1.08307421e+01]],

       [[ 2.91590972e-01,  7.77872076e-01, -1.99358616e-01,
         -7.84018052e-01,  1.35017147e+00, -3.54883289e-01,
          1.55131320e+00,  3.81996667e+00, -4.78766308e-01,
         -1.27649108e+00,  3.42685621e+00,  4.61556801e-01,
          5.81845890e+00, -1.09658919e+01],
        [ 3.53932535e-01,  7.68574302e-01, -2.20086772e-01,
         -7.25974074e-01,  1.35456507e+00, -2.74141435e-01,
          1.44690465e+00,  3.80681045e+00, -6.65719028e-01,
         -1.27487103e+00,  3.64899165e+00,  9.68696414e-02,
          3.65012540e+00,  8.81573825e-01],
        [ 4.15867987e-01,  7.56143947e-01, -2.41341472e-01,
         -6.65234258e-01,  1.35329320e+00, -2.31533342e-01,
          1.58336914e+00,  3.77945835e+00, -8.50084341e-01,
         -1.33474670e+00,  3.79324010e+00, -2.74566680e-01,
          2.15871807e+00,  1.33889074e+01],
        [ 4.77538038e-01,  7.40933632e-01, -2.63515669e-01,
         -6.01503620e-01,  1.34499927e+00, -1.88936365e-01,
          1.81813903e+00,  3.77779882e+00, -1.01088613e+00,
         -1.36364826e+00,  4.03878679e+00, -7.55687030e-01,
          3.28165422e+00,  1.38103651e+01]],

       [[ 4.77538038e-01,  7.40933632e-01, -2.63515669e-01,
         -6.01503620e-01,  1.34499927e+00, -1.88936365e-01,
          1.81813903e+00,  7.32957556e+00,  2.04627130e-01,
          6.27166078e-01,  6.17835070e+00,  1.49421668e+01,
         -1.32966853e+01, -1.57176678e+00],
        [ 5.98193595e-01,  7.42187629e-01, -2.49536073e-01,
         -4.41617012e-01,  1.66526137e+00, -4.06772666e-01,
          1.84158173e+00,  7.40521754e+00, -3.02029066e-02,
          9.16356063e-01,  1.28625460e+01,  1.43944973e+01,
         -1.40618413e+01,  4.88467592e+00],
        [ 7.18886371e-01,  7.40412496e-01, -2.36072368e-01,
         -2.22973662e-01,  1.73940691e+00, -6.48710749e-01,
          1.83839988e+00,  7.38062218e+00, -1.75125885e-01,
          7.48945923e-01,  1.33351279e+01, -1.45966592e+00,
         -1.43718016e+01, -1.40867193e+01],
        [ 8.39340540e-01,  7.36161710e-01, -2.24720803e-01,
         -1.06356517e-02,  1.68658078e+00, -8.48463122e-01,
          1.47582001e+00,  7.36684540e+00, -3.87020597e-01,
          6.52333724e-01,  1.26872985e+01, -4.47689511e+00,
         -1.20152991e+01, -1.38531007e+01]]]
)

tf = 0.8 # just now is tf = 2
destination = .35 * 4
unit_repeat = 2
repeat = 1
N = 4
mode_seqs = [
    RIGHT_STANCE, SPACE_STANCE, LEFT_STANCE, SPACE_STANCE,
    FLIP, RIGHT_STANCE, SPACE_STANCE, 
    DOUBLE_SUPPORT,
             ] 
n_mode = len(mode_seqs)

flip_initial = np.concatenate([run_initial_unit for _ in range(unit_repeat)], axis=0).reshape(n_mode * repeat, N, 14)
for i in range(1, unit_repeat):
  flip_initial[i*N: (i+1)*N, :, 0] += i * (flip_initial[3, -1, 0] - flip_initial[0, 0, 0])

for i in range(N):
  ang = 2 * np.pi / N * i
  flip_initial[4, i, 2] -= ang

flip_initial = flip_initial.reshape(n_mode * repeat * N, 14)

In [134]:
x_traj_run_flip_g= PiecewisePolynomial.ZeroOrderHold(
                    np.linspace(0, 0.1 * flip_initial.shape[0], flip_initial.shape[0]), 
                    flip_initial.T
                    )
vis(x_traj_run_flip_g, 0.1 * flip_initial.shape[0]+1, time_step=1.0)

In [136]:
#solver = TrajectoryOptimizationSolution(n_foot=6)
#x_traj_run_flip, u_traj_run_flip, prog = solver.solve(N, 
#                                    mode_seqs, 
#                                    repeat, 
#                                    flip_initial,
#                                     mu=5, 
#                                     tf=tf, 
#                                     destination=destination, 
#                                     iters=3e4,
#                                     test=False,
#                                     dest_cost=True,
#                                     backflip=True,
#                                     clockwise=True
#                                     )

In [121]:
vis(x_traj_run_flip, tf, 0.1)

In [ ]:
#%tb

import importlib
import traj_optim
import fsm_utils

importlib.reload(traj_optim)
importlib.reload(fsm_utils)
from traj_optim import TrajectoryOptimizationSolution, read_csv
from fsm_utils import (
    LEFT_STANCE, RIGHT_STANCE, SPACE_STANCE, DOUBLE_SUPPORT
)


solver = TrajectoryOptimizationSolution(n_foot=6)
N = 6
mode_seqs = [LEFT_STANCE, RIGHT_STANCE, LEFT_STANCE, SPACE_STANCE, DOUBLE_SUPPORT]
#mode_seqs = [RIGHT_STANCE, SPACE_STANCE, RIGHT_STANCE]
repeat = 1
n_mode = len(mode_seqs)

initial_state = np.array(
              [0.00000000,0.80000000,0.00000000,
              -0.64350111,1.28700222,-0.64350111,1.28700222,
              0.00000000,0.00000000,0.00000000,0.00000000,
              0.00000000,0.00000000,0.00000000]
      )
initial_guesses = np.concatenate([initial_state for _ in range(n_mode * N * repeat)]).reshape(n_mode * N * repeat, 14)
initial_guesses[N:2*N, 2] = np.linspace(initial_state[2], initial_state[2] + 2 * 3.14159, N )

tf = 1.0 # just now is tf = 2
destination = -1

x_traj_flip, u_traj, prog = solver.solve(N, 
                                    mode_seqs, 
                                    repeat, 
                                    initial_guesses,
                                     mu=30,  # to 10 later
                                     tf=tf, 
                                     destination=destination, 
                                     iters=4e4,
                                     test=False,
                                     backflip=True,
                                     clockwise=False
                                     )

In [ ]:
vis(x_traj_flip, tf, time_step=0.1)

## Test One

In [ ]:
from pydrake.systems.pyplot_visualizer import PyPlotVisualizer
from pydrake.all import PiecewisePolynomial
from pydrake.geometry import MeshcatAnimation

visualizer = PyPlotVisualizer()

initial_state = np.array(
              [0.00000000,0.80000000,0.00000000,
              -0.64350111,1.28700222,-0.64350111,1.28700222,
              0.00000000,0.00000000,0.00000000,0.00000000,
              0.00000000,0.00000000,0.00000000]
      )
n_mode = 1
N = 8
repeat = 1
initial_guesses = np.concatenate([initial_state for _ in range(n_mode * N * repeat)]).reshape(n_mode * N * repeat, 14)
initial_guesses[:N, 2] = np.linspace(initial_state[2], initial_state[2] + 2 * 3.14159, N )


x0 = np.array(
        [-1.42628695e-01, 7.91008947e-01, -1.42999388e-01,
          -7.89749727e-01, 1.31650877e+00, -2.43590062e-01, 1.04394874e+00, 
          0, 0, 0, 
          0, 0, 0, 0]
      )
xflip = np.array(
          [-1.42628695e-01, 7.91008947e-01, -1.42999388e-01,
         -2.43590062e-01, 1.04394874e+00,  -7.89749727e-01, 1.31650877e+00, 
          0, 0, 0, 
          0, 0, 0, 0]
)

x2 = x0.copy()
x2[2] += 1.5
#x2[3:7] += 3.14

x3 = x2.copy()
x3[2] += 1.5

x4 = x3.copy()
x4[2] += 1.5

file_name = "planar_walker.urdf"
builder = DiagramBuilder()
scene_graph = builder.AddSystem(SceneGraph())
plant = MultibodyPlant(0.0)
plant.RegisterAsSourceForSceneGraph(scene_graph)
Parser(plant=plant).AddModels(file_name)

plant.WeldFrames(
            plant.world_frame(),
            plant.GetBodyByName("base").body_frame(),
            RigidTransform.Identity()
        )

plant.Finalize()
plant_context = plant.CreateDefaultContext()
plant.SetPositionsAndVelocities(plant_context, x0.reshape(-1,1))

n_q = plant.num_positions()
n_v = plant.num_velocities()
n_u = plant.num_actuators()


traj00 = np.vstack(
                    [x0.reshape(-1,14),
                    x2.reshape(-1,14),
                    x3.reshape(-1,14),
                    x4.reshape(-1,14),
                    xflip.reshape(-1,14),
                    xflip.reshape(-1,14),
                    ]
                    )
traj00[:, 0] = np.linspace(x0[0], 3, 6)

x_traj00 = PiecewisePolynomial.ZeroOrderHold([0, 1, 2, 3, 4, 5], 
                                   traj00.T
                                           )

x_traj_flipping = PiecewisePolynomial.ZeroOrderHold(np.arange(n_mode * N * repeat),
                                                        initial_guesses.T)

x_traj_source = builder.AddSystem(TrajectorySource(x_traj_flipping))

demux = builder.AddSystem(Demultiplexer(np.array([n_q, n_v])))
to_pose = builder.AddSystem(MultibodyPositionToGeometryPose(plant))

builder.Connect(x_traj_source.get_output_port(), demux.get_input_port())
builder.Connect(demux.get_output_port(0), to_pose.get_input_port())
builder.Connect(to_pose.get_output_port(), scene_graph.get_source_pose_port(plant.get_source_id()))


MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)


diagram = builder.Build()
diagram.set_name("diagram")

ani = MeshcatAnimation()
simulator = Simulator(diagram)
simulator.Initialize()
simulator.set_target_realtime_rate(1)
simulator.AdvanceTo(10);


## Hopping

In [ ]:
#%tb

import importlib
import traj_optim
import fsm_utils

importlib.reload(traj_optim)
importlib.reload(fsm_utils)
from traj_optim import TrajectoryOptimizationSolution, read_csv
from fsm_utils import (
    LEFT_STANCE, RIGHT_STANCE, SPACE_STANCE, DOUBLE_SUPPORT
)


solver = TrajectoryOptimizationSolution(n_foot=6)
N = 4
mode_seqs = [RIGHT_STANCE, SPACE_STANCE] #, LEFT_STANCE, RIGHT_STANCE]
repeat = 2
tf = 0.8 # just now is tf = 2
destination = 1.2

initial_state = np.array(
    [0.00000000,0.80000000,0.00000000,
              -0.64350111,1.28700222,-0.64350111,1.28700222,
              0.20000000,0.00000000,0.00000000,0.00000000,
              0.00000000,0.00000000,0.00000000]
      )
initial_guesses = np.concatenate([initial_state for _ in range(n_mode * N * repeat)]).reshape(n_mode * N * repeat, 14)
initial_guesses[:, 0] = np.linspace(initial_state[0], destination, n_mode * N * repeat)

x_traj, u_traj, prog = solver.solve(N, 
                                    mode_seqs, 
                                    repeat, 
                                    initial_states,
                                     mu=1, 
                                     tf=tf, 
                                     destination=destination, 
                                     iters=2e4,
                                     test=False)

In [ ]:
vis(x_traj, tf, 0.1)